# Select the top 50 companies on the Stock Exchange of Thailand with Quantitative Value Strategy.
#### "Value investing" means investing in the stocks that are cheapest relative to common measures of business value.

For this project, we're going to build an investing strategy that selects stocks with the best value metrics below:
Price-to-sales ratio
Price-to-book ratio
Price-to-earnings ratio
Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
Enterprise Value divided by Gross Profit (EV/GP)

From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. which is an index of the top 50 companies on the Stock Exchange of Thailand (SET).

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
# import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from bs4 import BeautifulSoup

## List of Stocks with wikipedia

As mentioned before, we need to retrieve a list of stocks from this link: https://en.wikipedia.org/wiki/SET50_Index_and_SET100_Index before proceeding.

In [2]:
def list_set50_symbols():
    request = pd.read_html('https://en.wikipedia.org/wiki/SET50_Index_and_SET100_Index')
    stocks = request[0]['SET Symbol'].unique().tolist()
    set50_stock_symbol = []
    for symbol in range(len(stocks)):
        companie = stocks[symbol]
        set50_stock_symbol.append(str(companie) + ".BK")
    return set50_stock_symbol

## Create function to get the metric of key statistics.
We'll start by building a function that retrieve a data from yahoo finance. 

In [3]:
metric_key_statistics_aliases = {
    'Market Cap (intraday)': 'market_cap',
    'Beta (5Y Monthly)': 'beta',
    '52 Week High 3': '52_week_high',
    '52 Week Low 3': '52_week_low',
    '50-Day Moving Average 3': '50_day_ma',
    '200-Day Moving Average 3': '200_day_ma',
    'Avg Vol (3 month) 3': 'avg_vol_3m',
    'Avg Vol (10 day) 3': 'avg_vol_10d',
    'Shares Outstanding 5': 'shares_outstanding',
    'Float 8': 'float',
    '% Held by Insiders 1': 'held_by_insiders',
    '% Held by Institutions 1': 'held_by_institutions',
    'Short Ratio (Jan 30, 2023) 4': 'short_ratio',
    'Payout Ratio 4': 'payout_ratio',
    'Profit Margin': 'profit_margin',
    'Operating Margin (ttm)': 'operating_margin',
    'Return on Assets (ttm)': 'return_on_assets',
    'Return on Equity (ttm)': 'return_on_equity',
    'Revenue (ttm)': 'revenue',
    'Revenue Per Share (ttm)': 'revenue_per_share',
    'Gross Profit (ttm)': 'gross_profit',
    'EBITDA ': 'ebitda',
    'Net Income Avi to Common (ttm)': 'net_income',
    'Diluted EPS (ttm)': 'eps',
    'Total Cash (mrq)': 'total_cash',
    'Total Cash Per Share (mrq)': 'cash_per_share',
    'Total Debt (mrq)': 'total_debt',
    'Total Debt/Equity (mrq)': 'debt_to_equity',
    'Current Ratio (mrq)': 'current_ratio',
    'Book Value Per Share (mrq)': 'book_value_per_share',
    'Operating Cash Flow (ttm)': 'operating_cash_flow',
    'Levered Free Cash Flow (ttm)': 'levered_free_cash_flow'
}

def scrap_data_yfinance_key_statistics(ticker, metric_key_statistics_aliases):
    
    datas = {}
    url  = f"https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}" 
   
    user_agent_headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36"}

    page = requests.get(url, headers=user_agent_headers)
    soup = BeautifulSoup(page.content, "html.parser")
    sections = soup.find_all('section', {'data-test': 'qsp-statistics'})

    for section in sections:
        rows = section.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 2:
               metric = cols[0].text.strip()
               if metric in metric_key_statistics_aliases:
                  datas[metric_key_statistics_aliases[metric]] = cols[1].text.strip()
    return datas

In [4]:
metric_key_quote_aliases = {
    'Previous Close': 'prev_close',
    'Open': 'open',
    'Bid': 'bid',
    'Ask': 'ask',
    'Day\'s Range': 'day_range',
    '52 Week Range' : '52_w_range',
    'Volume': 'volume',
    'Avg. Volume': 'avg_volume',
    'Market Cap': 'market_cap',
    'Beta (5Y Monthly)': 'beta_5y_monthly',
    'PE Ratio (TTM)': 'pe_ratio_ttm',
    'EPS (TTM)': 'eps_ttm',
    'Earnings Date': 'earnings_date',
    'Forward Dividend & Yield': 'forward_dividend_yield',
    'Ex-Dividend Date': 'ex_dividend_date',
    '1y Target Est': '1y_target_est'
}

def scrap_data_yfinance_quote(ticker, metric_key_quote_aliases):
    datas = {}
    url  = f"https://finance.yahoo.com/quote/{ticker}?p={ticker}" 
    user_agent_headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36"}

    page = requests.get(url, headers=user_agent_headers)
    soup = BeautifulSoup(page.content, "html.parser")
    left_divs = soup.find_all('div', {'data-test': 'left-summary-table'})
    right_divs = soup.find_all('div', {'data-test': 'right-summary-table'})

    # Create an empty list to store the combined elements
    combined_elements = []

    # Append elements from left_divs to combined_elements
    for element in left_divs:
        combined_elements.append(element)
    
    # Append elements from right_divs to combined_elements
    for element in right_divs:
        combined_elements.append(element)

    for div in combined_elements:
        rows = div.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 2:
                metric = cols[0].text.strip()
                if metric in metric_key_quote_aliases:
                   datas[metric_key_quote_aliases[metric]] = cols[1].text.strip()
    return datas

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [5]:
quote_dataframe = pd.DataFrame(columns = ['Ticker', 'Market Cap', 'PE Ratio (TTM)', 'Number of Shares to Buy'])
symbols = list_set50_symbols()

for symbol in symbols:
    data = scrap_data_yfinance_quote(symbol, metric_key_quote_aliases)
    series = pd.Series({ "Ticker": symbol, "Market Cap": data['market_cap'], "PE Ratio (TTM)": data['pe_ratio_ttm'], "Number of Shares to Buy": 'N/A'})                     
    quote_dataframe = pd.concat([series.to_frame().T, quote_dataframe], ignore_index=True)

In [6]:
df = quote_dataframe.sort_values(by="Market Cap", ascending=True)
quote_dataframe

,Ticker,Market Cap,PE Ratio (TTM),Number of Shares to Buy
0,WHA.BK,71.446B,14.94,N/A
1,TU.BK,62.372B,15.22,N/A
2,TTB.BK,154.391B,8.83,N/A
3,TRUE.BK,198.675B,N/A,N/A
4,TOP.BK,105.549B,65.62,N/A
5,TLI.BK,114.5B,12.66,N/A
6,TISCO.BK,77.062B,10.67,N/A
7,TIDLOR.BK,59.561B,18.60,N/A
8,SCGP.BK,155.618B,34.86,N/A
9,SCC.BK,348B,12.79,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. To do this, we will use the `portfolio_input` function that we created in our momentum project. I have included this function below.

In [7]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [ ]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [ ]:
position_size = float(portfolio_size) / len(quote_dataframe.index)
for i in range(0, len(quote_dataframe['Ticker'])):
    quote_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / quote_dataframe['Price'][i])
quote_dataframe

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [ ]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [ ]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [ ]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()